Import Libraries

In [ ]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import Model
import cv2
import os
import tensorflow as tf

Load and preprocess Data

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), 1)
        img = cv2.resize(img,(224,224))
        img = preprocess_input(img)
        if img is not None:
            images.append(img)
    return images

In [ ]:
corona = load_images_from_folder("./COVID-19-master/X-Ray Image DataSet/Covid-19")
no_find = load_images_from_folder("./COVID-19-master/X-Ray Image DataSet/No_findings")
pneumonia = load_images_from_folder("./COVID-19-master/X-Ray Image DataSet/Pneumonia")
total = corona + no_find + pneumonia


key = {0 : "corona", 1 : "nofind", 2 : "pneumonia"}
label0 = []
for j in range(len(corona)):
  label0.append(0)
label1 = []
for j in range(len(no_find)):
  label1.append(1)
label2 = []
for j in range(len(pneumonia)):
  label2.append(2)
label = label0 + label1 + label2

dataset = np.array(total)
label = np.array(label)

Feature Extracting

In [ ]:
modeldn = tf.keras.applications.DenseNet201()
modeldn = Model(inputs=modeldn.inputs, outputs=modeldn.layers[-2].output)

xtrain, xtest, ytrain, ytest = train_test_split(dataset, label, test_size=0.2, random_state=85)

feature_extractor= modeldn.predict(xtrain)
xtrain = feature_extractor.reshape(feature_extractor.shape[0], -1)

Classification

In [ ]:
model = xgb.XGBClassifier(booster='gbtree',
                           learning_rate=0.15,
                           max_depth=2,
                           min_child_weight=2,
                           n_estimators=300,
                           n_jobs=1,
                           objective='multi:softmax',
                           reg_lambda=0,
                           num_class=3)
model.fit(xtrain, ytrain)
X_test_feature = modeldn.predict(xtest)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)
prediction = model.predict(X_test_features)
print("Accuracy = ", metrics.accuracy_score(ytest, prediction))

Accuracy =  0.9244444444444444


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(ytest, prediction)
print(cm)
print(classification_report(ytest, prediction))